# Building Interactive Visualizations in Jupyter Notebook


## What's Jupyter Notebook?
Jupyter Notebook (formerly IPython notebook) is a web application for scientific computation. jupyter **supports over 40 languages** (called kernals) including R and Matlab. Jupyter is growing in popularity and has become a standard tool for data scientists and researchers.

Python has a wealth of really fantastic libraries for analyzing and plotting data, including matplotlib, numpy and pandas. However, the plots are generated as static images. 

**Wouldn't it be more interesting have interactive plots?**


## Bokeh
For this demonstration, we'll use the [Bokeh](http://bokeh.pydata.org/en/latest/), pronounced "Bow (as in crossbow) Keh (as in kettle). We first need to tell  Bokeh to load its JavaScript component, BokehJS, using the ```output_notebook()``` function.

In [1]:
from bokeh.plotting import output_notebook, show

output_notebook()

Loading BokehJS ...

## Sample Data
A samll set of data, ```autompg```, available in the ```bokeh.sampledata``` package. ```autompg``` is in the ```pandas.dataframe``` format which you can think of as a database table or spreadsheet.

In [33]:
from bokeh.sampledata.autompg import autompg as df

#display first 5 elements of dataframe
df[0:5]

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


### Charts
Bokeh provides high level statistical plots such as bar charts, horizon plots and time series using the ```bokeh.charts``` interface. 

The interface is geared to be extremely simple to use in conjunction with the [Pandas](http://pandas.pydata.org/) data structures library (```Series``` or ```DataFrame```).

In the first plot below, we use the ```Bar``` chart to show the average miles per gallon for a given cylinder type. Note that we specify the ```mean``` aggregation type (default is sum).

In [3]:
from bokeh.charts import Bar

# group by cyl, apply aggregate average function to mpg column
barchart = Bar(df, label='cyl', values="mpg", agg='mean', title="Average MPG By Cycle")

show(barchart)

In [29]:
from bokeh.charts import Scatter

scatter = Scatter(df, x='mpg', y='hp', color='cyl', 
                  title="MPG vs Horsepower", xlabel="Miles Per Gallon",
                  ylabel="Horsepower")

show(scatter)

## Bokeh Linking
Bokeh plots can be displayed in rows with their interactions linked.

In [5]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,wheel_zoom,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300, title="|x|")
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300, x_range=left.x_range, title="x^2")
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

## Bokeh Widget
Widgets provide User Interface (UI) components such as sliders and buttons based on Bootstrap, making your bokeh visualizations dynamic. There are two ways to program widget functionality:

* Use the ```CustomJS``` callback. This works in standalone HTML documents
* Use the ```bokeh serve``` to start the Bokeh server and set up event handlers with ```.on_change```

In [32]:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import RangeSlider
from bokeh.client import push_session
from bokeh.plotting import curdoc

# open a session to keep our local document in sync with server
session = push_session(curdoc())

def range_slider_handler(attr, old, new):
    print("previous value: %s" % old)
    print("new value: %s" % new)
    
range_slider = RangeSlider(start=-25, end=25, range=(-20,20), step=1, title="X range")
range_slider.on_change("start", range_slider_handler)
range_slider.on_change("end", range_slider_handler)

show(widgetbox(range_slider))





Let's see what happens when we connect a widget to a bokeh chart. In this example, we'll select the ```cyl``` value to display on the scatter chart.

## Bokeh Custom Tools
If you're building apps around bokeh, you might want to customize some of the tools. Here's an example of how to do that.

In [34]:
from bokeh.core.properties import Instance
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, Tool, CustomJS
from bokeh.plotting import figure

JS_CODE = """
import * as p from "core/properties"
import {GestureTool, GestureToolView} from "models/tools/gestures/gesture_tool"

export class DrawToolView extends GestureToolView

  # this is executed when the pan/drag event starts
  _pan_start: (e) ->
    @model.source.data = {x: [], y: []}

  # this is executed on subsequent mouse/touch moves
  _pan: (e) ->
    frame = @plot_model.frame
    canvas = @plot_view.canvas

    vx = canvas.sx_to_vx(e.bokeh.sx)
    vy = canvas.sy_to_vy(e.bokeh.sy)
    if not frame.contains(vx, vy)
      return null

    x = frame.x_mappers['default'].map_from_target(vx)
    y = frame.y_mappers['default'].map_from_target(vy)

    @model.source.data.x.push(x)
    @model.source.data.y.push(y)
    @model.source.trigger('change')

  # this is executed then the pan/drag ends
  _pan_end: (e) -> return null

export class DrawTool extends GestureTool
  default_view: DrawToolView
  type: "DrawTool"

  tool_name: "Drag Span"
  icon: "bk-tool-icon-lasso-select"
  event_type: "pan"
  default_order: 12

  @define { source: [ p.Instance ] }
"""

class DrawTool(Tool):
    __implementation__ = JS_CODE
    source = Instance(ColumnDataSource)

source = ColumnDataSource(data=dict(x=[], y=[]))

# plot = figure(x_range=(0,10), y_range=(0,10), tools=[DrawTool(source=source)])
plot = Scatter(df, x='mpg', y='hp', color='cyl',
                  title="Auto MPG", xlabel="Miles Per Gallon",
                  ylabel="Horsepower", tools=[DrawTool(source=source), "save"])
# plot.title.text ="Drag to draw on the plot"
plot.line('x', 'y', source=source)

show(plot)

## Plotly
[Plotly](https://plot.ly/)  is an enterprise application providing advanced graphing and data analysis within the browser. The python library and associated javascript is open source. By default the service uses the ```plot.ly``` account and servers  to host data but you can also run without a server or with a paid enterprise service on your corporate network.